I. Programming basics

##### Declaration, definition and initialization 
* 声明式告诉编译器某个东西的名称和类型，但略去细节

In [23]:
extern int x;

extern ?

In [24]:
std::size_t numDigits(int number);

In [25]:
class Widget;

In [28]:
template<typename T>

* 定义式的任务是提供编译器一些声明式遗漏的细节。对对象而言，定义式是编译器为对象拨发内存的地点；对function或function template而言，定义式提供了代码本体。对class或class template而言，定义式列出他们的成员

In [30]:
std::size_t numDigits(int number){
    std::size_t digits = 1;
    
    while((number/=10)!=0) ++digits;
    return digits;
}

In [33]:
class Widget{
public:
    Widget();
    ~Widget();
};

* 初始化是“给予对象初值”的过程。default构造函数是一个可被调用而不带任何实参，不然没有参数，不然每个参数都有缺省值

###### 初始化参数列表
初始化类的成员有两种方式，一是使用初始化列表，二是在构造函数体内进行赋值操作。

对于对象类型，用初始化列表更高效，因为省去了调用默认构造函数的过程，直接使用拷贝构造函数。必须使用初始化列表的情况:
* 常量成员，因为常量只能初始化不能赋值，所以必须放在初始化列表里面
* 引用类型，引用必须在定义的时候初始化，并且不能重新赋值，所以也要写在初始化列表里面
* 没有默认构造函数的类类型，因为使用初始化列表可以不必调用默认构造函数来初始化，而是直接调用拷贝构造函数初始化。

    * 拷贝构造函数 ?
    * explicit ?
    * 隐式类型转换 ?
    

In [1]:
#include <string>

class A {
public:
    A(){};
};

class B {
public:
    int value;
    bool flag;

    explicit B(int x = 0, bool b = true):value(x), flag(b){}; 
};

class C {
public:
    std::string value;
    
    explicit C(std::string x):value(x){};
};



?   



In [2]:
B bObj;

In [3]:
B bObj1(1, false);

In [4]:
class D {
public:
    std::string id;
    D(std::string id):id(id){};
}

bool checkId(const D& other){
    return true;
}

In [5]:
D d("test");
checkId(std::string("test"))

(bool) true


这里就做了隐式类型转换，把string转换成class D.

R

In [6]:
bool checkId2(const C& other){
    return true;
}

In [7]:
checkId2(std::string("test"));

ename: evalue

##### reference and pointer
    1. const pointer vs pointer to const

* const pointer, 指针是常量，不能改变值

In [20]:
int a = 10, b = 20;
int * const  pInt = &a;

In [3]:
//pInt = &b;  //variable 'pInt' declared const here

In [4]:
a = 20;
*pInt

(int) 20


* pointer to const, 指向的是常量，不能通过指针修改其值，但是可以直接改被指向值

In [5]:
const int * cpInt = &a;

In [6]:
a = 30;
*cpInt

(const int) 30


In [7]:
//*cpInt = 20;  //read-only variable is not assignable

    2. const point to reference

##### Statements and flow control

    1. loop

In [21]:
#include <vector>
#include <iostream>
using namespace std;

In [22]:
vector<int> nums = {1,2};

* Range-based for loop

The for-loop has another syntax, which is used exclusively with ranges:

for ( declaration : range ) statement;

Ranges are sequences of elements, including arrays, containers, and any other type supporting the functions __begin()__ and __end()__; 

In [12]:
for (int i: nums){
    cout << i << endl;
}

5
6


User prefix increment, in for loop, extra copy involved with the postfix. So prefix can be faster

In [9]:
for (int i=0; i<nums.size(); ++i){
    cout << nums[i];
}

* iterator

In [11]:
for (vector<int>::iterator it=nums.begin(); it!=nums.end(); it++){
    *it += 1;  
    cout << *it << endl;
}

5
6


* begin return const_iterator if vector object is const-qualified

In [7]:
const vector<int> cNums = {1,2};    

for (auto it=cNums.begin(); it!=cNums.end(); it++){
    // *it += 1;  //function 'operator*' which returns const-qualified type
    cout << *it;
}

    2. switch case

In [ ]:
int condition = 1;

In [2]:
switch(condition){
    case 1:
    case 2:
        cout << "condition is 1 or 2"<< endl;
    default:
        cout << "condition did not match";
}

condition is 1 or 2


##### Dynamic Memory 
    * 用处：
        C++ 是by default是pass-by-value，面对vector等大的存储结构，很低效，如何把local vector传递出去？不要在stack中建立数据，建立动态内存，在堆中创建
        new，delete直接管理堆，会产生很多潜在问题。引入智能指针，shared_ptr, weak_ptr, unique_ptr来管理。
    * 变量作用域：
        全局变量
        局部变量
        static
        动态分配
    * 内存类型：
        栈内存-- 函数中非static  （全局变量？）
        static内存 -- static变量
        heap-堆 -- 动态分配   
        
    1. new and delete

* new

nothrow, if failed to allocate memory, will not throw bad alloc，but return nullptr

In [20]:
int *dp;
dp = new int;

In [23]:
dp = new (nothrow) int[5];

In [24]:
if(dp == nullptr){
    cout << "Error: memory could not be allocated" << endl;
}

In [25]:
*dp = 10;

* delete

In [26]:
delete [] dp;

    2. smart pointer
* shared_ptr

In [2]:
#include <memory>

In [3]:
shared_ptr<vector<int>> p = make_shared<vector<int>>()

(std::shared_ptr<std::vector<int> > &) @0x7fd116876018


In [4]:
p->push_back(5)
shared_ptr<vector<int>> p2 = p

p2->push_back(10);
p2->at(1);

(std::shared_ptr<std::vector<int> > &) @0x7fd116876028



* unique ptr

In [5]:
unique_ptr<int[]> pU(new int[5]);

In [19]:
pU[0] = 10;